In [11]:
import deconvolution_methods as DM
import numpy as np
from sklearn.metrics import adjusted_mutual_info_score as AMI
from scipy.stats import spearmanr,pearsonr
import pandas as pd
def bin_variable(var1):   # bin with normalization

    var1=np.array(var1).astype(np.float)

    if abs(np.std(var1))>0.01:
        var1 = (var1 - np.mean(var1))/np.std(var1)
    else:
        var1 = (var1 - np.mean(var1))
    val1 = np.digitize(var1, np.histogram(var1, bins='fd')[1])
    #print(type(val1))
    #print((val1).shape())
    return val1

ajd_mi_bin=lambda x,y : AMI(bin_variable(x),bin_variable(y))


In [12]:
# load data
data_in_silico=pd.read_csv('net1_expression_data.tsv',sep='\t')
print(data_in_silico)

           G1        G2        G3        G4        G5        G6        G7  \
0    0.425448  0.017829  0.907989  0.448247  0.172324  0.273489  0.843766   
1    0.442400  0.050525  0.869368  0.445851  0.173311  0.274889  0.764049   
2    1.056847  0.208454  0.467448  0.505077  0.244883  0.208451  0.665355   
3    1.117226  0.003001  0.317654  0.387204  0.253792  0.179360  0.939244   
4    0.971068  0.001056  0.354651  0.474532  0.207718  0.102833  0.745871   
5    1.139386  0.122047  0.402465  0.348436  0.168614  0.255774  0.924055   
6    1.064869  0.140508  0.481763  0.474857  0.182643  0.112430  1.081748   
7    0.876117  0.073814  1.058292  0.730366  0.053656  0.175109  1.202648   
8    1.205966  0.153407  0.760861  0.655846  0.157731  0.141754  1.039769   
9    1.000675  0.078986  0.803631  0.838975  0.088306  0.151089  1.071518   
10   0.928186  0.227125  0.702909  0.867608  0.061720  0.185059  0.989553   
11   0.947588  0.212333  0.609245  0.796102  0.092966  0.222524  0.698124   

In [13]:
P_corr=np.corrcoef(data_in_silico.transpose())

In [4]:
print(P_corr.shape)
print(len(list(data_in_silico['G1'])))

(1643, 1643)
805


In [25]:
S_corr=np.ones((1643,1643))
AMI_fd=np.ones((1643,1643))
for idx_i,i in zip(range(len(data_in_silico.columns)),data_in_silico.columns):
    for idx_j,j in zip(range(len(data_in_silico.columns)),data_in_silico.columns):
        if i!=j:
            S_corr[idx_i,idx_j]=spearmanr(list(data_in_silico[i]),list(data_in_silico[j]))[0]
            AMI_fd[idx_i,idx_j]=ajd_mi_bin(list(data_in_silico[i]),list(data_in_silico[j]))

In [35]:
print(S_corr)

[[ 1.         -0.00225177 -0.21438459 ...,  0.02601459 -0.06175255
   0.12834364]
 [-0.00225177  1.         -0.13923577 ..., -0.17931808  0.34836044
   0.0686446 ]
 [-0.21438459 -0.13923577  1.         ...,  0.43508113 -0.20182508
  -0.05472033]
 ..., 
 [ 0.02601459 -0.17931808  0.43508113 ...,  1.         -0.41291694
   0.09282972]
 [-0.06175255  0.34836044 -0.20182508 ..., -0.41291694  1.         -0.12056149]
 [ 0.12834364  0.0686446  -0.05472033 ...,  0.09282972 -0.12056149  1.        ]]


In [34]:
print(AMI_fd)

[[ 1.          0.01878533  0.01644456 ...,  0.01831519  0.00912871
   0.00339487]
 [ 0.01878533  1.          0.03327592 ...,  0.0381582   0.04501715
   0.01816231]
 [ 0.01644456  0.03327592  1.         ...,  0.06667567  0.03245433
   0.01355769]
 ..., 
 [ 0.01831519  0.0381582   0.06667567 ...,  1.          0.06838425
   0.0034866 ]
 [ 0.00912871  0.04501715  0.03245433 ...,  0.06838425  1.          0.02390381]
 [ 0.00339487  0.01816231  0.01355769 ...,  0.0034866   0.02390381  1.        ]]


In [32]:
S_corr=np.loadtxt('S_corrM.nptxt')
np.savetxt('AMI_fd.csv',AMI_fd,delimiter=",")

In [20]:
import deconvolution_methods as DM
??DM

In [16]:
# %load ND.py
def ND(mat,beta=0.90,alpha=1,control=0):
    '''
    This is a python implementation/translation of network deconvolution by MIT-KELLIS LAB
    
    
     LICENSE: MIT-KELLIS LAB
    
    
     AUTHORS:
        Algorithm was programmed by Soheil Feizi.
        Paper authors are S. Feizi, D. Marbach,  M. M?©dard and M. Kellis
    Python implementation: Gideon Rosenthal
    
    REFERENCES:
       For more details, see the following paper:
        Network Deconvolution as a General Method to Distinguish
        Direct Dependencies over Networks
        By: Soheil Feizi, Daniel Marbach,  Muriel Médard and Manolis Kellis
        Nature Biotechnology
    
    --------------------------------------------------------------------------
     ND.m: network deconvolution
    --------------------------------------------------------------------------
    
    DESCRIPTION:
    
     USAGE:
        mat_nd = ND(mat)
        mat_nd = ND(mat,beta)
        mat_nd = ND(mat,beta,alpha,control)
    
    
     INPUT ARGUMENTS:
     mat           Input matrix, if it is a square matrix, the program assumes
                   it is a relevance matrix where mat(i,j) represents the similarity content
                   between nodes i and j. Elements of matrix should be
                   non-negative.
     optional parameters:
     beta          Scaling parameter, the program maps the largest absolute eigenvalue
                   of the direct dependency matrix to beta. It should be
                   between 0 and 1.
     alpha         fraction of edges of the observed dependency matrix to be kept in
                   deconvolution process.
     control       if 0, displaying direct weights for observed
                   interactions, if 1, displaying direct weights for both observed and
                   non-observed interactions.
    
     OUTPUT ARGUMENTS:
    
     mat_nd        Output deconvolved matrix (direct dependency matrix). Its components
                   represent direct edge weights of observed interactions.
                   Choosing top direct interactions (a cut-off) depends on the application and
                   is not implemented in this code.
    
     To apply ND on regulatory networks, follow steps explained in Supplementary notes
     1.4.1 and 2.1 and 2.3 of the paper.
     In this implementation, input matrices are made symmetric.
    
    **************************************************************************
     loading scaling and thresholding parameters
    '''
    import scipy.stats.mstats as stat
    from numpy import linalg as LA


    if beta>=1 or beta<=0:
        print 'error: beta should be in (0,1)'
      
    if alpha>1 or alpha<=0:
            print 'error: alpha should be in (0,1)';
     
    
    '''
    ***********************************
     Processing the inut matrix
     diagonal values are filtered
    '''
    
    n = mat.shape[0]
    np.fill_diagonal(mat, 0)
    
    '''
    Thresholding the input matrix
    '''
    y =stat.mquantiles(mat[:],prob=[1-alpha])
    th = mat>=y
    mat_th=mat*th;

    '''
    making the matrix symetric if already not
    '''
    mat_th = (mat_th+mat_th.T)/2

    
    '''
    ***********************************
    eigen decomposition
    '''
    print 'Decomposition and deconvolution...'

    Dv,U = LA.eigh(mat_th) 
    D = np.diag((Dv))
    lam_n=np.abs(np.min(np.min(np.diag(D)),0))
    lam_p=np.abs(np.max(np.max(np.diag(D)),0))

    
    m1=lam_p*(1-beta)/beta
    m2=lam_n*(1+beta)/beta
    m=max(m1,m2)
    
    #network deconvolution
    for i in range(D.shape[0]):
        D[i,i] = (D[i,i])/(m+D[i,i])
    
    mat_new1 = np.dot(U,np.dot(D,LA.inv(U)))
    
                    
    '''
    
    ***********************************
     displying direct weights
    '''
    if control==0:
        ind_edges = (mat_th>0)*1.0;
        ind_nonedges = (mat_th==0)*1.0;
        m1 = np.max(np.max(mat*ind_nonedges));
        m2 = np.min(np.min(mat_new1));
        mat_new2 = (mat_new1+np.max(m1-m2,0))*ind_edges+(mat*ind_nonedges);
    else:
        m2 = np.min(np.min(mat_new1));
        mat_new2 = (mat_new1+np.max(-m2,0));
    
    
    '''
    ***********************************
     linearly mapping the deconvolved matrix to be between 0 and 1
    '''
    m1 = np.min(np.min(mat_new2));
    m2 = np.max(np.max(mat_new2));
    mat_nd = (mat_new2-m1)/(m2-m1);


    return mat_nd



In [17]:
ND_S= ND(S_corr)
ND_P= ND(P_corr)
ND_AMI=ND(AMI_fd)

Decomposition and deconvolution...
Decomposition and deconvolution...
Decomposition and deconvolution...


In [18]:
np.savetxt('ND_S.csv',ND_S,delimiter=",")
np.savetxt('ND_P.csv',ND_P,delimiter=",")
np.savetxt('ND_AMI.csv',ND_AMI,delimiter=",")

In [ ]:
PartCorr_S= DM.deconvolution_methods(3,S_corr)
PartCorr_P= DM.deconvolution_methods(3,P_corr)
PartCorr_AMI=DM.deconvolution_methods(3,AMI_fd)

In [25]:
#Reprocessing
PartCorr_S=np.absolute(np.nan_to_num(PartCorr_S))
PartCorr_P=np.absolute(np.nan_to_num(PartCorr_P))
PartCorr_AMI=np.absolute(np.nan_to_num(PartCorr_AMI))

print(PartCorr_S)

[[ 0.          0.13000715  0.27215561 ...,  0.11628633  0.05761709
   0.74997131]
 [ 0.13000715  0.          0.48133178 ...,  0.00307377  0.07546168
   0.18489732]
 [ 0.27215561  0.48133178  0.         ...,  0.2458947   0.14180642
   1.88940185]
 ..., 
 [ 0.11628633  0.00307377  0.2458947  ...,  0.          0.06462468
   0.56901616]
 [ 0.05761709  0.07546168  0.14180642 ...,  0.06462468  0.          0.14999466]
 [ 0.74997131  0.18489732  1.88940185 ...,  0.56901616  0.14999466  0.        ]]


In [26]:
np.savetxt('PartCorr_S.csv',PartCorr_S,delimiter=",")
np.savetxt('PartCorr_P.csv',PartCorr_P,delimiter=",")
np.savetxt('PartCorr_AMI.csv',PartCorr_AMI,delimiter=",")

In [28]:
data_in_silico.transpose().to_csv('data_in_silico.csv',sep=',',header=False,index=False)